# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:

from backbone.utils.wfo_utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30
COMMISSION = 7e-4

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

#
limited_testing_start_date = Timestamp('2022-01-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2023-01-01 00:00:00', tz='UTC')

In [4]:

groups = [
    'Forex_Indicator',
    'CryptoCross_grp',
    'Crypto_group',
    'Energies_group',
    'Forex_group',
    'Indices_group',
    'Stocks_group'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbols = mt5.symbols_get()

# tickers = [symbol.path.split('\\')[1] for symbol in symbols if ('CryptoCross_grp' in symbol.path or 'Crypto_group' in symbol.path or 'Energies_group' in symbol.path or 'Indices_group' in symbol.path) and symbol.spread < 10]
tickers = ['USTECm']

print(tickers)

intervals = [
    mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['USTECm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
USTECm


In [6]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date


intervals_start_dates


print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16388: Timestamp('2021-10-27 16:00:00+0000', tz='UTC')}
2022-01-01 00:00:00+00:00
2023-01-01 00:00:00+00:00


# Estrategia

In [9]:
from backbone.channel_strategy import Channel

STRATEGY = Channel

# Analisis preliminar

In [10]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, np.unique(intervals), strategies
))

stats_per_symbol = {}

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    # try:
    print(ticker, interval)
    
    if ticker not in stats_per_symbol.keys():
        stats_per_symbol[ticker] = {}
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

    df_stats, stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN
    )

    performance = pd.concat([performance, df_stats])
    stats_per_symbol[ticker][interval] = stats

    # except Exception as e:
    #     print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


USTECm 16388


In [11]:

filter_performance = performance.sort_values(by=['return/dd'], ascending=[False])#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Channel,USTECm,16388,0.363365,4.4071,10440.70995,8.42723,188 days 16:00:00,34.0,0.307869,50,0.0584,49.329073,10440.70995,362 days,0.522959,1.838074


In [13]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

    
    df_stats = run_strategy(
        strategy=STRATEGY,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )
    
    break


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


# WFO

In [14]:
from backbone.utils.wfo_utils import optimization_function

lookback_bars = 1800
validation_bars = 180
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
all_opt_params = {}

params = {
    'sma_period' : list(np.arange(9, 24, 3)),
    'atr_multiplier' : [1.5, 2, 2.5, 3],
}

for index, row in filter_performance.iterrows():
    
    try:
        ticker = row.ticker
        interval = row.interval
        
        print(ticker, interval)
        
        prices = symbols[ticker][interval]

        wfo_stats, df_stats, opt_params = run_wfo(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            prices=prices,
            initial_cash=INITIAL_CASH,
            commission=COMMISSION,
            margin=MARGIN,
            optim_func=optimization_function,
            params=params,
            lookback_bars=lookback_bars,
            warmup_bars=warmup_bars,
            validation_bars=validation_bars,
            plot=True
        )
                
        if ticker not in all_opt_params.keys():
            all_opt_params[ticker] = {}

        all_opt_params[ticker][interval] = opt_params

        all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    except Exception as e:
        print(f'No se pudo ejecutar para el ticker {ticker}: {e}')
    
all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

USTECm 16388


In [15]:

performance = pd.DataFrame()

wfo_stats_per_symbol = {}

for index, row in filter_performance.iterrows():
    try:
        ticker = row.ticker
        interval = row.interval
        print(ticker, interval)
        
        prices = symbols[ticker][interval]
        params = all_opt_params[ticker][interval]
        
        if ticker not in wfo_stats_per_symbol.keys():
            wfo_stats_per_symbol[ticker] = {}

        df_stats, wfo_stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            opt_params=params, 
            plot=True
        )

        wfo_stats_per_symbol[ticker][interval] = wfo_stats

        performance = pd.concat([performance, df_stats])

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


USTECm 16388


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

In [16]:
filtered_wfo_performance = performance[
    (performance['return/dd'] > 2)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Channel,USTECm,16388,0.842218,28.936454,12893.645423,9.372112,245 days 20:00:00,33.834586,0.695707,133,0.153843,52.579632,12893.645423,1038 days,3.087506,13.664153


# Montecarlo

In [26]:
def montecarlo_statistics_simulation(
    trade_history,
    equity_curve,
    n_simulations,
    initial_equity,
    threshold_ruin=0.85,
    return_raw_curves=False,
    percentiles=None,
):

    # Renombro las columnas
    print('hola')
    trade_history = trade_history.rename(columns={"ExitTime": "Date"})
    trade_history = trade_history[["Date", "PnL"]]

    equity_curve = (
        equity_curve.reset_index()
        .rename(columns={"index": "Date"})[["Date", "Equity"]]
        .sort_values(by="Date")
    )

    trade_history["Date"] = pd.to_datetime(trade_history["Date"])
    equity_curve["Date"] = pd.to_datetime(equity_curve["Date"])

    # joineo los dfs por fechas

    full_df = pd.merge(equity_curve, trade_history, on="Date", how="left")

    full_df = full_df[~full_df["PnL"].isna()]

    # Porcentaje de ganancia

    full_df["pct"] = full_df["PnL"] / full_df["Equity"].shift(1)

    # Parámetros iniciales

    n_steps = len(trade_history)
    mean_return = full_df["pct"].mean()
    std_return = full_df["pct"].std()

    drawdowns_pct = []  # Lista para almacenar los drawdowns en porcentaje
    final_returns_pct = []  # Lista para almacenar los retornos finales en porcentaje
    ruin_count = 0  # Contador de simulaciones que alcanzan la ruina
    ruin_threshold = (
        initial_equity * threshold_ruin
    )  # Umbral de ruina en términos de equidad

    # Función para calcular el drawdown máximo en porcentaje

    def max_drawdown(equity_curve):
        running_max = np.maximum.accumulate(equity_curve)
        drawdown = (equity_curve - running_max) / running_max
        return np.min(drawdown) * 100  # Convertir el drawdown a porcentaje

    # Simulaciones de Montecarlo

    for _ in range(n_simulations):
        # Generar retornos aleatorios con media y desviación estándar de los históricos

        # random_returns = mean_return + std_return * np.random.standard_t(15, size=n_steps)
        random_returns = np.random.normal(
            loc=mean_return, scale=std_return, size=n_steps
        )

        # Calcular la curva de equidad acumulada

        synthetic_equity_curve = initial_equity * np.cumprod(1 + random_returns)

        # Calcular drawdown y almacenarlo en porcentaje

        dd_pct = max_drawdown(synthetic_equity_curve)
        drawdowns_pct.append(dd_pct)

        # Calcular el retorno acumulado porcentual y almacenarlo

        final_return_pct = (
            synthetic_equity_curve[-1] / initial_equity - 1
        ) * 100  # Retorno final en porcentaje
        final_returns_pct.append(final_return_pct)

        # Verificar si la equidad cae por debajo del umbral de ruina en algún punto

        if np.any(synthetic_equity_curve <= ruin_threshold):
            ruin_count += 1
    # Crear un DataFrame separado para los drawdowns y los retornos acumulados en porcentaje

    df_drawdowns = pd.DataFrame({"Drawdown (%)": drawdowns_pct})
    df_final_returns_pct = pd.DataFrame({"Final Return (%)": final_returns_pct})

    # Calcular las estadísticas usando df.describe() para cada DataFrame

    if not percentiles:
        drawdown_stats = df_drawdowns.describe()
        return_stats = df_final_returns_pct.describe()
    else:
        drawdown_stats = df_drawdowns.describe(percentiles=percentiles)
        return_stats = df_final_returns_pct.describe(percentiles=percentiles)
    # Calcular el riesgo de ruina

    risk_of_ruin = ruin_count / n_simulations

    # Agregar el riesgo de ruina a las estadísticas de drawdown

    drawdown_stats.loc["Risk of Ruin"] = risk_of_ruin

    # Combinar las métricas de drawdowns y retornos porcentuales

    combined_stats = pd.concat([drawdown_stats, return_stats], axis=1)
    if return_raw_curves:
        return combined_stats, df_drawdowns, df_final_returns_pct
    
    return combined_stats

In [27]:

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=INITIAL_CASH, 
        threshold_ruin=0.6, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

Procesando ticker: USTECm
hola


In [28]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


,drawdown_USTECm
count,100000.000000
mean,-8.433026
std,3.752199
min,-32.709314
10%,-13.598755
25%,-10.457900
50%,-7.662645
75%,-5.660287
90%,-4.371046
max,-1.205536


,return_USTECm
count,100000.000000
mean,5.817577
std,10.570846
min,-31.518363
10%,-7.370685
25%,-1.535655
50%,5.374627
75%,12.577799
90%,19.603795
max,61.206374


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
